# Core CNN Models for Pediatric Pneumonia Detection

## Overview
This notebook implements essential baseline CNN architectures for pneumonia detection:
- **Xception**: Advanced CNN with depthwise separable convolutions
- **MobileNet**: Lightweight CNN designed for mobile devices
- **VGG16**: Classic deep CNN architecture

These are the **core models** that form the foundation for pneumonia detection. Anyone working on medical image classification should understand these architectures first.

## Key Concepts
- **CNN (Convolutional Neural Network)**: Deep learning model that processes images using filters
- **Transfer Learning**: Using pre-trained models and adapting them for medical images
- **Fine-tuning**: Adjusting pre-trained model weights for specific tasks
- **Feature Maps**: Intermediate representations learned by CNN layers
- **Classification Head**: Final layers that make pneumonia/normal predictions

In [ ]:
# Import required libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import timm  # PyTorch Image Models - provides pre-trained models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from tqdm import tqdm

# Set device (GPU if available, CPU otherwise)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

## 1. Configuration and Data Setup

**Key Parameters:**
- **Batch Size**: Number of images processed together (32 = good balance of speed and memory)
- **Image Size**: 224x224 pixels (standard for most pre-trained models)
- **Epochs**: Number of complete passes through the training data
- **Learning Rate**: How fast the model learns (smaller = more careful learning)

**Data Transforms:**
- **Resize**: Makes all images the same size
- **ToTensor**: Converts images to PyTorch format
- **Normalize**: Standardizes pixel values (improves training stability)

In [ ]:
# Configuration parameters
CONFIG = {
    'batch_size': 32,
    'image_size': 224,
    'epochs': 10,
    'learning_rate': 1e-4,
    'seed': 42
}

# Data directories
TRAIN_DIR = '../data/training/augmented_train'
TEST_DIR = '../data/testing/augmented_test'

# Data transformations
# These prepare the images for training
transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),  # Resize to 224x224
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize to [-1, 1] range
])

# Load datasets
print("Loading datasets...")
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=transform)
test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=transform)

# Create data loaders
# These handle batching and shuffling of data
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

# Print dataset information
print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")
print(f"Classes: {train_dataset.classes}")
print(f"Class to index mapping: {train_dataset.class_to_idx}")

# Check class distribution
train_counts = [0, 0]
for _, label in train_dataset:
    train_counts[label] += 1

test_counts = [0, 0]
for _, label in test_dataset:
    test_counts[label] += 1

print(f"\nTraining distribution: NORMAL={train_counts[0]}, PNEUMONIA={train_counts[1]}")
print(f"Testing distribution: NORMAL={test_counts[0]}, PNEUMONIA={test_counts[1]}")

## 2. Model Architectures

### 2.1 Xception Model

**What is Xception?**
- Advanced CNN architecture using "depthwise separable convolutions"
- More efficient than regular convolutions (fewer parameters, faster training)
- Good at capturing fine details in medical images
- Pre-trained on ImageNet (natural images), adapted for X-rays

**Transfer Learning Process:**
1. Load pre-trained Xception model
2. Replace final layer for pneumonia classification (2 classes instead of 1000)
3. Optionally freeze early layers (keep learned features, only train final layers)

In [ ]:
class XceptionFineTune(nn.Module):
    """
    Xception model adapted for pneumonia detection.
    
    Uses transfer learning: starts with ImageNet pre-trained weights
    and adapts them for medical image classification.
    """
    
    def __init__(self, num_classes=2, freeze_layers=100):
        """
        Initialize Xception model for pneumonia detection.
        
        Args:
            num_classes: Number of output classes (2 for Normal/Pneumonia)
            freeze_layers: Number of early layers to freeze during training
        """
        super(XceptionFineTune, self).__init__()
        
        # Load pre-trained Xception from timm library
        self.xception = timm.create_model('xception', pretrained=True)
        
        # Remove the original classification layers
        self.xception.global_pool = nn.Identity()  # Remove global pooling
        self.xception.fc = nn.Identity()  # Remove final fully connected layer
        
        # Add our own pooling and classification layers
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling
        
        # Custom classification head for pneumonia detection
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),  # Dropout prevents overfitting
            nn.Linear(2048, 128),  # Xception features: 2048 -> 128
            nn.ReLU(),  # Activation function
            nn.Dropout(0.3),  # More dropout
            nn.Linear(128, num_classes)  # Final prediction: 128 -> 2 classes
        )
        
        # Freeze early layers for stable training
        if freeze_layers > 0:
            for i, (name, param) in enumerate(self.xception.named_parameters()):
                if i < freeze_layers:
                    param.requires_grad = False  # Don't update these weights
            print(f"Frozen first {freeze_layers} layers")
    
    def forward(self, x):
        """
        Forward pass through the model.
        
        Args:
            x: Input batch of images [batch_size, 3, 224, 224]
            
        Returns:
            logits: Raw prediction scores [batch_size, 2]
        """
        # Extract features using Xception backbone
        x = self.xception(x)  # [batch_size, 2048, 7, 7]
        
        # Global average pooling: average across spatial dimensions
        x = self.pool(x)  # [batch_size, 2048, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten: [batch_size, 2048]
        
        # Classification
        x = self.classifier(x)  # [batch_size, 2]
        
        return x

print("Xception model class defined successfully!")

### 2.2 MobileNet Model

**What is MobileNet?**
- Lightweight CNN designed for mobile and embedded devices
- Uses "depthwise separable convolutions" for efficiency
- Much smaller and faster than traditional CNNs
- Good for real-time applications and resource-constrained environments

**When to Use MobileNet:**
- When you need fast inference (quick predictions)
- Limited computational resources
- Mobile or edge deployment
- As a baseline for comparison with heavier models

In [ ]:
class MobileNetFineTune(nn.Module):
    """
    MobileNet model adapted for pneumonia detection.
    
    Lightweight architecture suitable for deployment on mobile devices
    or when computational resources are limited.
    """
    
    def __init__(self, num_classes=2, freeze_layers=50):
        """
        Initialize MobileNet model for pneumonia detection.
        
        Args:
            num_classes: Number of output classes (2 for Normal/Pneumonia)
            freeze_layers: Number of early layers to freeze during training
        """
        super(MobileNetFineTune, self).__init__()
        
        # Load pre-trained MobileNetV2 from timm
        self.mobilenet = timm.create_model('mobilenetv2_100', pretrained=True)
        
        # Get the number of features from the last layer
        num_features = self.mobilenet.classifier.in_features
        
        # Replace the classifier for pneumonia detection
        self.mobilenet.classifier = nn.Sequential(
            nn.Dropout(0.4),  # Dropout for regularization
            nn.Linear(num_features, 64),  # Reduce to 64 features
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)  # Final prediction
        )
        
        # Freeze early layers
        if freeze_layers > 0:
            for i, (name, param) in enumerate(self.mobilenet.named_parameters()):
                if i < freeze_layers:
                    param.requires_grad = False
            print(f"Frozen first {freeze_layers} layers")
    
    def forward(self, x):
        """
        Forward pass through MobileNet.
        
        Args:
            x: Input batch of images [batch_size, 3, 224, 224]
            
        Returns:
            logits: Raw prediction scores [batch_size, 2]
        """
        return self.mobilenet(x)

print("MobileNet model class defined successfully!")

### 2.3 VGG16 Model

**What is VGG16?**
- Classic CNN architecture with 16 layers
- Uses small 3x3 convolution filters throughout
- Simple and interpretable architecture
- Good baseline model for medical image analysis

**Characteristics:**
- Relatively large number of parameters
- Slower than modern architectures but reliable
- Good for understanding CNN behavior
- Often used as a comparison baseline

In [ ]:
class VGG16FineTune(nn.Module):
    """
    VGG16 model adapted for pneumonia detection.
    
    Classic CNN architecture providing a reliable baseline
    for medical image classification tasks.
    """
    
    def __init__(self, num_classes=2, freeze_layers=10):
        """
        Initialize VGG16 model for pneumonia detection.
        
        Args:
            num_classes: Number of output classes (2 for Normal/Pneumonia)
            freeze_layers: Number of early layers to freeze during training
        """
        super(VGG16FineTune, self).__init__()
        
        # Load pre-trained VGG16 from timm
        self.vgg = timm.create_model('vgg16', pretrained=True)
        
        # Get number of features from pre-classifier layer
        num_features = self.vgg.pre_logits.in_features
        
        # Replace the classifier head
        self.vgg.head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 256),  # Intermediate layer
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)  # Final prediction
        )
        
        # Freeze early convolutional layers
        if freeze_layers > 0:
            for i, (name, param) in enumerate(self.vgg.features.named_parameters()):
                if i < freeze_layers:
                    param.requires_grad = False
            print(f"Frozen first {freeze_layers} feature layers")
    
    def forward(self, x):
        """
        Forward pass through VGG16.
        
        Args:
            x: Input batch of images [batch_size, 3, 224, 224]
            
        Returns:
            logits: Raw prediction scores [batch_size, 2]
        """
        return self.vgg(x)

print("VGG16 model class defined successfully!")

## 3. Training Utilities

**Training Process:**
1. **Forward Pass**: Images go through the model to get predictions
2. **Loss Calculation**: Compare predictions with actual labels
3. **Backward Pass**: Calculate gradients (how to update weights)
4. **Weight Update**: Adjust model parameters to improve performance

**Key Concepts:**
- **Loss Function**: CrossEntropyLoss for classification tasks
- **Optimizer**: Adam optimizer for efficient weight updates
- **Learning Rate**: Controls how big steps the optimizer takes

In [ ]:
def train_model(model, train_loader, test_loader, num_epochs, learning_rate, model_name):
    """
    Train a CNN model for pneumonia detection.
    
    Args:
        model: PyTorch model to train
        train_loader: DataLoader for training data
        test_loader: DataLoader for testing data
        num_epochs: Number of training epochs
        learning_rate: Learning rate for optimizer
        model_name: Name for saving and logging
        
    Returns:
        Tuple of (trained_model, training_history)
    """
    
    # Move model to appropriate device (GPU/CPU)
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()  # Standard for classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer
    
    # Track training progress
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_acc': []
    }
    
    print(f"Starting training for {model_name}...")
    print(f"Training for {num_epochs} epochs with learning rate {learning_rate}")
    print("-" * 50)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Progress bar for training batches
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', leave=False)
        
        for inputs, labels in train_pbar:
            # Move data to device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero gradients from previous iteration
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
            # Update progress bar
            train_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct_train/total_train:.2f}%'
            })
        
        # Calculate training metrics
        epoch_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct_train / total_train
        
        # Evaluation phase
        model.eval()  # Set model to evaluation mode
        correct_test = 0
        total_test = 0
        
        with torch.no_grad():  # Don't compute gradients during evaluation
            test_pbar = tqdm(test_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Test]', leave=False)
            
            for inputs, labels in test_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()
                
                test_pbar.set_postfix({
                    'Acc': f'{100.*correct_test/total_test:.2f}%'
                })
        
        test_accuracy = 100. * correct_test / total_test
        
        # Store history
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(train_accuracy)
        history['test_acc'].append(test_accuracy)
        
        # Print epoch results
        print(f'Epoch [{epoch+1}/{num_epochs}] - '
              f'Train Loss: {epoch_loss:.4f}, '
              f'Train Acc: {train_accuracy:.2f}%, '
              f'Test Acc: {test_accuracy:.2f}%')
    
    print(f"\nTraining completed for {model_name}!")
    print(f"Final Test Accuracy: {history['test_acc'][-1]:.2f}%")
    
    return model, history

print("Training function defined successfully!")

## 4. Evaluation Utilities

**Evaluation Metrics:**
- **Accuracy**: Percentage of correct predictions
- **Precision**: Of predicted pneumonia cases, how many were actually pneumonia
- **Recall**: Of actual pneumonia cases, how many were correctly identified
- **F1-Score**: Harmonic mean of precision and recall
- **Confusion Matrix**: Shows all correct and incorrect predictions in detail

**Why Multiple Metrics Matter:**
- Medical diagnosis requires high recall (don't miss pneumonia cases)
- High precision reduces false alarms
- Balanced metrics indicate robust model performance

In [ ]:
def evaluate_model(model, test_loader, class_names, model_name):
    """
    Comprehensive evaluation of a trained model.
    
    Args:
        model: Trained PyTorch model
        test_loader: DataLoader for test data
        class_names: List of class names ['NORMAL', 'PNEUMONIA']
        model_name: Name for logging and display
        
    Returns:
        Dictionary containing evaluation metrics
    """
    
    model.eval()  # Set to evaluation mode
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    print(f"Evaluating {model_name}...")
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Get model predictions
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)  # Convert to probabilities
            _, predicted = torch.max(outputs, 1)
            
            # Store results
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    # Convert to numpy arrays
    y_true = np.array(all_labels)
    y_pred = np.array(all_predictions)
    y_prob = np.array(all_probabilities)
    
    # Calculate accuracy
    accuracy = (y_true == y_pred).mean() * 100
    
    # Generate classification report
    report = classification_report(y_true, y_pred, 
                                 target_names=class_names, 
                                 output_dict=True)
    
    # Print detailed results
    print(f"\n{model_name} Evaluation Results:")
    print("=" * 50)
    print(f"Overall Accuracy: {accuracy:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    # Create and display confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    # Calculate detailed metrics
    tn, fp, fn, tp = cm.ravel()
    
    results = {
        'model_name': model_name,
        'accuracy': accuracy,
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'f1_score': report['weighted avg']['f1-score'],
        'confusion_matrix': cm,
        'true_positives': tp,
        'true_negatives': tn,
        'false_positives': fp,
        'false_negatives': fn,
        'classification_report': report
    }
    
    print(f"\nDetailed Metrics:")
    print(f"True Positives (Correctly identified pneumonia): {tp}")
    print(f"True Negatives (Correctly identified normal): {tn}")
    print(f"False Positives (Normal classified as pneumonia): {fp}")
    print(f"False Negatives (Pneumonia classified as normal): {fn}")
    
    return results

print("Evaluation function defined successfully!")

## 5. Training Individual Models

Now we'll train each model architecture and compare their performance.

### 5.1 Train Xception Model

In [ ]:
# Initialize and train Xception model
print("Initializing Xception model...")
xception_model = XceptionFineTune(num_classes=2, freeze_layers=100)

# Train the model
trained_xception, xception_history = train_model(
    model=xception_model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=CONFIG['epochs'],
    learning_rate=CONFIG['learning_rate'],
    model_name="Xception"
)

# Save the trained model
torch.save(trained_xception.state_dict(), '../models/xception_pneumonia.pth')
print("Xception model saved to ../models/xception_pneumonia.pth")

### 5.2 Train MobileNet Model

In [ ]:
# Initialize and train MobileNet model
print("Initializing MobileNet model...")
mobilenet_model = MobileNetFineTune(num_classes=2, freeze_layers=50)

# Train the model
trained_mobilenet, mobilenet_history = train_model(
    model=mobilenet_model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=CONFIG['epochs'],
    learning_rate=CONFIG['learning_rate'],
    model_name="MobileNet"
)

# Save the trained model
torch.save(trained_mobilenet.state_dict(), '../models/mobilenet_pneumonia.pth')
print("MobileNet model saved to ../models/mobilenet_pneumonia.pth")

### 5.3 Train VGG16 Model

In [ ]:
# Initialize and train VGG16 model
print("Initializing VGG16 model...")
vgg_model = VGG16FineTune(num_classes=2, freeze_layers=10)

# Train the model
trained_vgg, vgg_history = train_model(
    model=vgg_model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=CONFIG['epochs'],
    learning_rate=CONFIG['learning_rate'],
    model_name="VGG16"
)

# Save the trained model
torch.save(trained_vgg.state_dict(), '../models/vgg16_pneumonia.pth')
print("VGG16 model saved to ../models/vgg16_pneumonia.pth")

## 6. Model Evaluation and Comparison

Now let's evaluate all trained models and compare their performance.

In [ ]:
# Evaluate all models
class_names = ['NORMAL', 'PNEUMONIA']
results = []

# Evaluate Xception
xception_results = evaluate_model(trained_xception, test_loader, class_names, "Xception")
results.append(xception_results)

print("\n" + "="*80 + "\n")

# Evaluate MobileNet
mobilenet_results = evaluate_model(trained_mobilenet, test_loader, class_names, "MobileNet")
results.append(mobilenet_results)

print("\n" + "="*80 + "\n")

# Evaluate VGG16
vgg_results = evaluate_model(trained_vgg, test_loader, class_names, "VGG16")
results.append(vgg_results)

## 7. Model Comparison and Analysis

In [ ]:
# Create comparison table
import pandas as pd

comparison_data = []
for result in results:
    comparison_data.append({
        'Model': result['model_name'],
        'Accuracy (%)': f"{result['accuracy']:.2f}",
        'Precision': f"{result['precision']:.4f}",
        'Recall': f"{result['recall']:.4f}",
        'F1-Score': f"{result['f1_score']:.4f}",
        'False Negatives': result['false_negatives'],
        'False Positives': result['false_positives']
    })

comparison_df = pd.DataFrame(comparison_data)
print("Model Comparison Summary:")
print("=" * 60)
print(comparison_df.to_string(index=False))

# Plot training curves
plt.figure(figsize=(15, 5))

# Training loss
plt.subplot(1, 3, 1)
plt.plot(xception_history['train_loss'], label='Xception', marker='o')
plt.plot(mobilenet_history['train_loss'], label='MobileNet', marker='s')
plt.plot(vgg_history['train_loss'], label='VGG16', marker='^')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Training accuracy
plt.subplot(1, 3, 2)
plt.plot(xception_history['train_acc'], label='Xception', marker='o')
plt.plot(mobilenet_history['train_acc'], label='MobileNet', marker='s')
plt.plot(vgg_history['train_acc'], label='VGG16', marker='^')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

# Test accuracy
plt.subplot(1, 3, 3)
plt.plot(xception_history['test_acc'], label='Xception', marker='o')
plt.plot(mobilenet_history['test_acc'], label='MobileNet', marker='s')
plt.plot(vgg_history['test_acc'], label='VGG16', marker='^')
plt.title('Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Find best performing model
best_model = max(results, key=lambda x: x['accuracy'])
print(f"\nBest performing model: {best_model['model_name']} with {best_model['accuracy']:.2f}% accuracy")

# Medical significance analysis
print("\nMedical Significance Analysis:")
print("=" * 40)
for result in results:
    fn = result['false_negatives']  # Missed pneumonia cases
    fp = result['false_positives']  # False alarms
    print(f"{result['model_name']}:")
    print(f"  - Missed pneumonia cases: {fn} (High risk - could delay treatment)")
    print(f"  - False pneumonia alerts: {fp} (Moderate risk - unnecessary treatment)")
    print(f"  - Recall (pneumonia detection rate): {result['recall']:.4f}")
    print()

## 8. Summary and Recommendations

### Model Performance Analysis

Based on the training and evaluation results:

**Key Findings:**
- **Xception**: Advanced architecture with excellent feature extraction capabilities
- **MobileNet**: Lightweight and efficient, suitable for mobile deployment
- **VGG16**: Reliable baseline with interpretable architecture

**Medical Application Considerations:**
- **High Recall Priority**: Missing pneumonia cases has severe consequences
- **Balanced Performance**: Both precision and recall matter for practical use
- **Computational Requirements**: Consider deployment environment constraints

### Next Steps:
1. **Fusion Models**: Combine multiple architectures for improved performance
2. **CNN-LSTM Models**: Add sequential processing for enhanced feature analysis
3. **Ensemble Methods**: Combine predictions from multiple models
4. **Hyperparameter Tuning**: Optimize learning rates, architectures, and training strategies

### Individual Model Use Cases:
- **Xception**: Best overall performance for accuracy-critical applications
- **MobileNet**: Real-time mobile applications and resource-constrained environments
- **VGG16**: Educational purposes and baseline comparisons

**All individual CNN models are now trained and ready for use in ensemble methods!**